# This code attempts to check the validity of my jacobian layerwise method given a layer with biases. We will take the right answer as given by the 'easy' method, since it just computes straight jacobians

In [1]:
import numpy as np
import torch
import random

import matplotlib.pyplot as plt


import torch
from torch import nn, optim
from torch import load
from torch.nn import functional as F
from torch import autograd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#from ..easy_ntk import calculate_NTK
#from einops import rearrange

import time

import sys
from pathlib import Path

from numba import njit
from numba.typed import List

%matplotlib inline
%load_ext line_profiler
%load_ext memory_profiler

In [2]:
import copy

def _del_nested_attr(obj, names):
    """
    Deletes the attribute specified by the given list of names.
    For example, to delete the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'])
    """
    if len(names) == 1:
        delattr(obj, names[0])
    else:
        _del_nested_attr(getattr(obj, names[0]), names[1:])

def _set_nested_attr(obj, names, value):
    """
    Set the attribute specified by the given list of names to value.
    For example, to set the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'], value)
    """
    if len(names) == 1:
        setattr(obj, names[0], value)
    else:
        _set_nested_attr(getattr(obj, names[0]), names[1:], value)

def extract_weights(mod):
    """
    This function removes all the Parameters from the model and
    return them as a tuple as well as their original attribute names.
    The weights must be re-loaded with `load_weights` before the model
    can be used again.
    Note that this function modifies the model in place and after this
    call, mod.parameters() will be empty.
    """
    orig_params = tuple(mod.parameters())
    # Remove all the parameters in the model
    names = []
    for name, p in list(mod.named_parameters()):
        _del_nested_attr(mod, name.split("."))
        names.append(name)

    # Make params regular Tensors instead of nn.Parameter
    params = tuple(p.detach().requires_grad_() for p in orig_params)
    return params, names

def load_weights(mod, names, params):
    """
    Reload a set of weights so that `mod` can be used again to perform a forward pass.
    Note that the `params` are regular Tensors (that can have history) and so are left
    as Tensors. This means that mod.parameters() will still be empty after this call.
    """
    for name, p in zip(names, params):
        _set_nested_attr(mod, name.split("."), p)
        
def calculate_NTK(model,x,device='cpu',MODE='samples'):
    """
    INPUTS:
        model: torch.nn.Module 
        x: torch.Tensor
        device: 'cpu',
        MODE: 'minima'
    
    OUTPUTS:
        NTK: torch.Tensor
    
    Calculates the NTK for a model, p_dict a state dictionary, and x, a single tensor fed into the model
    
    The NTK is the grammian of the Jacobian of the model output to w.r.t. the weights of the model
    
    This function will output the NTK such that the minima matrix size is used. If the Jacobian is an NxM
    matrix, then the NTK is formulated so that if N < M; NTK is NxN. If M<N, then NTK is MxM.
    
    #EXAMPLE USAGE:
    device='cpu'
    model = MODEL() #a torch.nn.Module object 
    model.to(device)
    state_dict = model.state_dict()

    x_test = np.ones((100,1,28,28),dtype=np.float32)
    x_test = torch.from_numpy(x_test)

    NTK = calculate_NTK(model,x_test)
    """
    if not(MODE in ['minima','samples','params']):
        raise ValueError("MODE must be one of 'minima','samples','params'")
    
    x = x.to(device)
    x.requires_grad=False
    N = x.shape[0]
    M = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    #We need to create a clone of the model or else we make it unusable as part of the trickery 
    #to get pytorch to do what we want. Unforutantely, this exlcludes super big models. but, eh.
    model_clone = copy.deepcopy(model)
    
    params, names = extract_weights(model_clone)
    def model_ntk(*args,model=model_clone, names=names):
        params = tuple(args)
        load_weights(model, names, params)
        return model(x)
    
    Js = torch.autograd.functional.jacobian(model_ntk, tuple(params), create_graph=False, vectorize=True)
    
    Js = list(Js)
    #Js = [element for tupl in Js for element in tupl]
    #collapse the tensors
    for i,tensor in enumerate(Js):
        Js[i] = tensor.reshape(N,-1)
    
    J = torch.cat(Js,axis=1)
    
    if MODE=='minima':
        if N < M: #if datasize points is less than number of parameters:
            NTK = torch.matmul(J,J.T)

        if N >= M:#if number of parameters is less than datasize:
            NTK = torch.matmul(J.T,J)
    elif MODE=='samples':
        NTK = torch.matmul(J,J.T)
    elif MODE=='params':
        NTK = torch.matmul(J.T,J)
    
    return NTK

In [3]:
def relu(X,normalize=False):
    X = F.relu(X)
    if normalize:
        return np.sqrt(2*np.pi/(np.pi-1))*(X-1/np.sqrt(2*np.pi))
    else:
        return X
    

# #Identity
# def activation(x):
#     return x

# @njit
# def d_activation(x):
#     return np.ones(np.shape(x),dtype=np.float32) 


#Tanh
def activation(x):
    return torch.tanh(x)

@njit
def d_activation(x):
    return np.cosh(x)**-2

In [4]:
SEED = 1

In [5]:
def NTK_weights(m):
    if isinstance(m, nn.Linear):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]
    if isinstance(m, nn.Conv2d):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]

In [42]:
# Easy NTK expects one output alone
class dumb_small(torch.nn.Module):
    '''
    simple network for test cases
    
    
    It seems like bias vectors aren't trivially added.
    '''
    def __init__(self,):
        super(dumb_small, self).__init__()
        
        self.d1 = torch.nn.Linear(5,256,bias=True)
        self.d2 = torch.nn.Linear(256,256,bias=True)
        self.d3 = torch.nn.Linear(256,256,bias=True)
        self.d4 = torch.nn.Linear(256,1,bias=False)
        
    def forward(self, x_0):
        x_1 = activation(self.d1(x_0)) / np.sqrt(256)
        x_2 = activation(self.d2(x_1)) / np.sqrt(256)
        x_3 = activation(self.d3(x_2)) / np.sqrt(256)
        x_4 = self.d4(x_3)
        return x_4

In [43]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
device='cpu'

model_small = dumb_small()
model_small.to(device)
model_small.apply(NTK_weights)

x_test = np.random.normal(0,1,(10,5)).astype(np.float32)
x_test = torch.from_numpy(x_test)

torch.Size([256, 5])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([1, 256])


#### Jacobian autograd

In [44]:
NTK_easy = calculate_NTK(model_small,x_test).detach().numpy()

# Compare

In [45]:
print(NTK_easy)

[[1.6082418  0.78865474 1.4226712  0.96147966 0.82881945 1.0671781
  1.0429941  0.96164554 0.9174976  0.8706212 ]
 [0.78865474 1.5072196  0.75161815 1.131173   1.1097087  1.0987914
  1.0313911  0.86794126 0.9297676  0.8734958 ]
 [1.4226712  0.75161815 1.6992437  0.9873619  0.8369214  1.0702881
  1.038669   1.06961    1.001262   0.9081332 ]
 [0.96147966 1.131173   0.9873619  1.3857301  1.0011244  1.3634785
  1.1525166  0.94906884 1.036931   0.8774627 ]
 [0.82881945 1.1097087  0.8369214  1.0011244  1.4609115  0.9590928
  0.89021856 0.9942888  0.9264419  1.136893  ]
 [1.0671781  1.0987914  1.0702881  1.3634785  0.9590928  1.4511997
  1.2238289  0.9317817  1.0226927  0.85774785]
 [1.0429941  1.0313911  1.038669   1.1525166  0.89021856 1.2238289
  1.5406871  0.86796474 0.9619671  0.8749953 ]
 [0.96164554 0.86794126 1.06961    0.94906884 0.9942888  0.9317817
  0.86796474 1.3861303  1.1500888  0.998226  ]
 [0.9174976  0.9297676  1.001262   1.036931   0.9264419  1.0226927
  0.9619671  1.150088

# Can we get pytorch to give us the matrices with the correct kinds of derivative?

In [46]:
model_small.zero_grad()
x_4 = model_small(x_test)

In [47]:
#this method agrees between model layerwise and model small; meaning that the calculation is indepdent of those
#two models. the insinuation is somehting is wrong with both my methods for calculating,--- the same thing, since
#they agree with one another.

#in the future we would iterate over layers instead of like this...
layer_components_w1 = [] 
layer_components_w2 = []
layer_components_w3 = []
layer_components_w4 = []

layer_components_b1 = []
layer_components_b2 = []
layer_components_b3 = []


for output in x_4:
    model_small.zero_grad()
    
    output.backward(retain_graph=True)

    #Get the tensors
    w4_grad = model_small.d4.weight.grad.detach().numpy()
    w3_grad = model_small.d3.weight.grad.detach().numpy()
    w2_grad = model_small.d2.weight.grad.detach().numpy()
    w1_grad = model_small.d1.weight.grad.detach().numpy()
    b1_grad = model_small.d1.bias.grad.detach().numpy()
    b2_grad = model_small.d2.bias.grad.detach().numpy()
    b3_grad = model_small.d3.bias.grad.detach().numpy()

    #reshape and append. deep copy neccessary or else they are the same objects
    layer_components_w1.append(w1_grad.reshape(-1).copy())
    layer_components_w2.append(w2_grad.reshape(-1).copy())
    layer_components_w3.append(w3_grad.reshape(-1).copy())
    layer_components_w4.append(w4_grad.reshape(-1).copy())
    layer_components_b1.append(b1_grad.reshape(-1).copy())
    layer_components_b2.append(b2_grad.reshape(-1).copy())
    layer_components_b3.append(b3_grad.reshape(-1).copy())


In [48]:
layer_components_w1 = np.array(layer_components_w1)
layer_components_w2 = np.array(layer_components_w2)
layer_components_w3 = np.array(layer_components_w3)
layer_components_w4 = np.array(layer_components_w4)

layer_components_b1 = np.array(layer_components_b1)
layer_components_b2 = np.array(layer_components_b2)
layer_components_b3 = np.array(layer_components_b3)

In [49]:
autograd_NTK = layer_components_w1 @ layer_components_w1.T+\
    layer_components_w2 @ layer_components_w2.T+\
    layer_components_w3 @ layer_components_w3.T+\
    layer_components_w4 @ layer_components_w4.T+\
    layer_components_b1 @ layer_components_b1.T+\
    layer_components_b2 @ layer_components_b2.T+\
    layer_components_b3 @ layer_components_b3.T

In [50]:
autograd_NTK

array([[1.6082417 , 0.7886546 , 1.4226713 , 0.9614797 , 0.8288195 ,
        1.0671782 , 1.0429943 , 0.96164566, 0.91749746, 0.8706212 ],
       [0.7886546 , 1.5072201 , 0.7516182 , 1.131173  , 1.1097085 ,
        1.0987915 , 1.0313914 , 0.86794114, 0.9297677 , 0.8734957 ],
       [1.4226713 , 0.7516182 , 1.6992437 , 0.9873618 , 0.8369217 ,
        1.0702883 , 1.0386689 , 1.0696099 , 1.0012618 , 0.90813327],
       [0.9614797 , 1.131173  , 0.9873618 , 1.38573   , 1.0011241 ,
        1.3634782 , 1.1525165 , 0.9490689 , 1.0369308 , 0.8774627 ],
       [0.8288195 , 1.1097085 , 0.8369217 , 1.0011241 , 1.4609115 ,
        0.9590931 , 0.8902186 , 0.99428856, 0.9264417 , 1.136893  ],
       [1.0671782 , 1.0987915 , 1.0702883 , 1.3634782 , 0.9590931 ,
        1.451199  , 1.2238294 , 0.93178177, 1.0226927 , 0.8577478 ],
       [1.0429943 , 1.0313914 , 1.0386689 , 1.1525165 , 0.8902186 ,
        1.2238294 , 1.540687  , 0.86796474, 0.96196723, 0.87499523],
       [0.96164566, 0.86794114, 1.0696099

In [51]:
NTK_easy

array([[1.6082418 , 0.78865474, 1.4226712 , 0.96147966, 0.82881945,
        1.0671781 , 1.0429941 , 0.96164554, 0.9174976 , 0.8706212 ],
       [0.78865474, 1.5072196 , 0.75161815, 1.131173  , 1.1097087 ,
        1.0987914 , 1.0313911 , 0.86794126, 0.9297676 , 0.8734958 ],
       [1.4226712 , 0.75161815, 1.6992437 , 0.9873619 , 0.8369214 ,
        1.0702881 , 1.038669  , 1.06961   , 1.001262  , 0.9081332 ],
       [0.96147966, 1.131173  , 0.9873619 , 1.3857301 , 1.0011244 ,
        1.3634785 , 1.1525166 , 0.94906884, 1.036931  , 0.8774627 ],
       [0.82881945, 1.1097087 , 0.8369214 , 1.0011244 , 1.4609115 ,
        0.9590928 , 0.89021856, 0.9942888 , 0.9264419 , 1.136893  ],
       [1.0671781 , 1.0987914 , 1.0702881 , 1.3634785 , 0.9590928 ,
        1.4511997 , 1.2238289 , 0.9317817 , 1.0226927 , 0.85774785],
       [1.0429941 , 1.0313911 , 1.038669  , 1.1525166 , 0.89021856,
        1.2238289 , 1.5406871 , 0.86796474, 0.9619671 , 0.8749953 ],
       [0.96164554, 0.86794126, 1.06961  

In [68]:
np.allclose(NTK_easy,autograd_NTK,rtol=0.000001)

False

# Now show for CNNs

In [78]:
class small_CNN(torch.nn.Module):
    '''
    simple CNN to see if convolutional layers also behave this way.
    '''
    def __init__(self,):
        super(small_CNN, self).__init__()
        self.c1 = torch.nn.Conv2d(1,3,3,bias=False)
        self.d1 = torch.nn.Linear(3*26*26,1,bias=False)
    def forward(self, x_0):
        x_1 = activation(self.c1(x_0)) / np.sqrt(3*3*3)
        x_1 = x_1.view(len(x_1),-1)
        x_2 = self.d1(x_1)
        return x_2
    

In [79]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
device='cpu'

model_small = small_CNN()
model_small.to(device)
model_small.apply(NTK_weights)

#Reset the seed and 
# torch.manual_seed(SEED)
# random.seed(SEED)
# np.random.seed(SEED)


# model_layerwise = dumb_small_layerwise()
# model_layerwise.to(device)
# model_layerwise.apply(NTK_weights)

x_test = np.random.normal(0,1,(10,1,28,28)).astype(np.float32)
x_test = torch.from_numpy(x_test)

torch.Size([3, 1, 3, 3])
torch.Size([1, 2028])


In [80]:
model_small.zero_grad()
x_4 = model_small(x_test)

In [81]:
#this method agrees between model layerwise and model small; meaning that the calculation is indepdent of those
#two models. the insinuation is somehting is wrong with both my methods for calculating,--- the same thing, since
#they agree with one another.

#in the future we would iterate over layers instead of like this...
layer_components_w1 = [] #and we can make these arrays since we will know their size and slice into them
# and we would only do this one array element at a time.
layer_components_c1 = []



for output in x_4:
    model_small.zero_grad()
    
    output.backward(retain_graph=True)
    
    c1_grad = model_small.c1.weight.grad.detach().numpy()
    w1_grad = model_small.d1.weight.grad.detach().numpy()
    
    layer_components_w1.append(w1_grad.reshape(-1).copy())
    layer_components_c1.append(c1_grad.reshape(-1).copy())

    
layer_components_w1 = np.array(layer_components_w1)
layer_components_c1 = np.array(layer_components_c1)

autograd_NTK = layer_components_w1 @ layer_components_w1.T+\
    layer_components_c1 @ layer_components_c1.T

In [84]:
NTK_easy = calculate_NTK(model_small,x_test).detach().numpy()

In [82]:
autograd_NTK

array([[185.849     ,   4.753747  ,  35.843     , -10.557724  ,
        -27.003904  ,  -6.7686734 ,   4.9796295 ,  -3.574995  ,
         60.676533  ,  18.215563  ],
       [  4.753747  , 127.567924  , -25.370993  ,   0.65108633,
        -18.790497  ,   4.472741  ,  11.494589  , -18.128845  ,
          1.4599648 ,  14.607652  ],
       [ 35.843     , -25.370993  , 162.48811   , -20.927635  ,
        -24.948666  , -25.155651  ,  20.570183  ,  17.539974  ,
         15.7003765 , -15.755165  ],
       [-10.557724  ,   0.65108633, -20.927635  , 120.6505    ,
         20.385773  ,  13.934135  , -22.998585  , -32.750645  ,
        -16.121796  ,  25.182693  ],
       [-27.003904  , -18.790497  , -24.948666  ,  20.385773  ,
        213.54944   ,   0.3297494 , -58.5591    ,  -5.195174  ,
        -37.370903  ,   5.855665  ],
       [ -6.7686734 ,   4.472741  , -25.155651  ,  13.934135  ,
          0.3297494 , 124.84943   , -20.54618   ,  12.445291  ,
         45.03541   ,  11.195108  ],
       [  

In [85]:
NTK_easy

array([[185.84906   ,   4.7537465 ,  35.842987  , -10.557731  ,
        -27.003891  ,  -6.7686663 ,   4.979627  ,  -3.5749938 ,
         60.676544  ,  18.215565  ],
       [  4.7537465 , 127.56792   , -25.371012  ,   0.65108544,
        -18.790497  ,   4.472741  ,  11.494592  , -18.128847  ,
          1.4599633 ,  14.607651  ],
       [ 35.842987  , -25.371012  , 162.48807   , -20.927633  ,
        -24.948662  , -25.155651  ,  20.570179  ,  17.539986  ,
         15.700378  , -15.755162  ],
       [-10.557731  ,   0.65108544, -20.927633  , 120.65047   ,
         20.385775  ,  13.93413   , -22.998564  , -32.75066   ,
        -16.121792  ,  25.182676  ],
       [-27.003891  , -18.790497  , -24.948662  ,  20.385775  ,
        213.54947   ,   0.3297524 , -58.559105  ,  -5.195169  ,
        -37.37091   ,   5.8556623 ],
       [ -6.7686663 ,   4.472741  , -25.155651  ,  13.93413   ,
          0.3297524 , 124.84941   , -20.546185  ,  12.4452915 ,
         45.03543   ,  11.195114  ],
       [  

In [94]:
np.allclose(NTK_easy,autograd_NTK,rtol=0.00001)

False